In [1]:
import pandas as pd
import math
import numpy as np
import itertools

In [2]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [3]:
stats = '/home/acer/github/nba/stats.csv'
games = '/home/acer/github/nba/games.csv'

In [165]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv(stats, header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('.','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('-','')
statsDF['MIN'] = statsDF['MIN'].replace({'\$': '', '-': ''}, regex=True)
#Drop the first line for the team name
statsDF.drop(0,inplace=True)

In [166]:
#The nba.com site changed its formatting so this is to remove non-existent rows

new_format_mask = 0
for index, row in statsDF.iterrows():
    if new_format_mask == 1:
        statsDF.drop(index, inplace=True)
        if row.iloc[0] == 'Totals:' and math.isnan(row.iloc[1]) == True:
            new_format_mask = 0 
        else:
            continue
    else:
        if row.iloc[0] == 'PLAYER' and row.iloc[1] != "MIN":
            new_format_mask += 1
        else:
            continue

for index, row in statsDF.iterrows():
    if row.iloc[0][3:4] == ":":
        statsDF.drop(index, inplace=True)

    

In [167]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'INACTIVE']
statsDF = statsDF.reset_index(drop=True)

In [168]:
for index, row in statsDF.iterrows():
    try: 
        if row.iloc[1].isalpha() and statsDF.iloc[index + 1][1].isalpha():
            home_team_length = index/2
            statsDF.drop(index, inplace=True)
            statsDF = statsDF.reset_index(drop=True)
            break
    except:
        continue

In [170]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if row.iloc[0].isalpha() == True and row.iloc[1].isalpha() == True:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Maurice Harkless', 'AlFarouq Aminu', 'Jusuf Nurkic', 'CJ McCollum', 'Damian Lillard', 'Pat Connaughton', 'Evan Turner', 'Ed Davis', 'Shabazz Napier', 'Zach Collins', 'Jake Layman', 'Meyers Leonard', 'Caleb Swanigan', 'Khris Middleton', 'Giannis Antetokounmpo', 'Thon Maker', 'Tony Snell', 'Malcolm Brogdon', 'DeAndre Liggins', 'Mirza Teletovic', 'Greg Monroe', 'Matthew Dellavedova', 'Jason Terry', 'John Henson', 'Sterling Brown', 'DJ Wilson']


In [176]:
#Setup dicts from the DF
statsDF_Dict = statsDF.T.to_dict().values()

In [183]:
dict_list = []
#Iterate through the statsDF and only return lines with stats or lines where players sat
for index, i in enumerate(statsDF_Dict):
    if index == 1 or index % 2 != 0:
        dict_list.append(i)
    

In [185]:
dict_list

[{'+/-': nan,
  '3P%': '1',
  '3PA': '50.0',
  '3PM': '4',
  'AST': '1',
  'BLK': '3',
  'DREB': '3',
  'FG%': '2',
  'FGA': '50.0',
  'FGM': '10',
  'FT%': '0',
  'FTA': '50.0',
  'FTM': '2',
  'MIN': '5',
  'OREB': '3',
  'PF': '13',
  'PLAYER': '37:50',
  'PTS': '1',
  'REB': '0',
  'STL': '0',
  'TOV': '3'},
 {'+/-': nan,
  '3P%': '2',
  '3PA': '33.3',
  '3PM': '3',
  'AST': '0',
  'BLK': '1',
  'DREB': '6',
  'FG%': '1',
  'FGA': '14.3',
  'FGM': '7',
  'FT%': '1',
  'FTA': '100',
  'FTM': '2',
  'MIN': '1',
  'OREB': '5',
  'PF': '5',
  'PLAYER': '25:35',
  'PTS': '-1',
  'REB': '2',
  'STL': '1',
  'TOV': '0'},
 {'+/-': nan,
  '3P%': '3',
  '3PA': '0.0',
  '3PM': '1',
  'AST': '2',
  'BLK': '1',
  'DREB': '11',
  'FG%': '0',
  'FGA': '41.2',
  'FGM': '17',
  'FT%': '5',
  'FTA': '75.0',
  'FTM': '4',
  'MIN': '7',
  'OREB': '6',
  'PF': '17',
  'PLAYER': '31:58',
  'PTS': '-7',
  'REB': '3',
  'STL': '3',
  'TOV': '0'},
 {'+/-': nan,
  '3P%': '5',
  '3PA': '50.0',
  '3PM': '6',


In [187]:
finalStats = pd.DataFrame(dict_list)
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')

In [188]:
finalStats

FTM   3P% 3PA TOV  PF REB 3PM       FG%      FGA OREB  \
Maurice Harkless        1  50.0   4   1   3   3   2      50.0       10    0   
AlFarouq Aminu          2  33.3   3   0   1   6   1      14.3        7    1   
Jusuf Nurkic            3   0.0   1   2   1  11   0      41.2       17    5   
CJ McCollum             5  50.0   6   2   2   3   3      50.0       18    0   
Damian Lillard         11  50.0   6   6   5   5   3      40.0       15    2   
Pat Connaughton         0  33.3   3   2   1   0   1      25.0        4    0   
Evan Turner             4  25.0   4   2   5   2   1      42.9        7    0   
Ed Davis                4   0.0   0   1   4  11   0       100        1    4   
Shabazz Napier          1   0.0   1   0   0   1   0      50.0        2    1   
Zach Collins           NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   
Jake Layman            NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   
Meyers Leonard         NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   
Caleb Swanigan         NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   
Khris Middleton         6   0.0   1   2   5   7   0      46.2       13    0   
Giannis Antetokounmpo   9  25.0   4   2   3   8   1      73.9       23    2   
Thon Maker              3   0.0   2   2   5   3   0       0.0        3    0   
Tony Snell              0  75.0   4   2   1   2   3      87.5        8    1   
Malcolm Brogdon         3   0.0   0   2   2   1   0      35.7       14    1   
DeAndre Liggins         0   0.0   1   1   1   1   0       0.0        2    0   
Mirza Teletovic         0  75.0   4   0   2   2   3      75.0        4    0   
Greg Monroe             0   0.0   0   0   0   0   0       0.0        3    0   
Matthew Dellavedova     0   0.0   2   1   0   2   0      20.0        5    0   
Jason Terry             0   0.0   0   0   0   0   0       0.0        0    0   
John Henson             1   0.0   0   2   4   9   0      60.0        5    3   
Sterling Brown         NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   
DJ Wilson              NA    NA  NA  NA  NA  NA  NA  DECISION  COACH'S   NA   

                        FT% FTA FGM DREB PTS    MIN +/- AST STL BLK  
Maurice Harkless       50.0   2   5    3  13  37:50   1   0   0   3  
AlFarouq Aminu          100   2   1    5   5  25:35  -1   2   1   0  
Jusuf Nurkic           75.0   4   7    6  17  31:58  -7   3   3   0  
CJ McCollum             100   5   9    3  26  37:03   1   1   0   2  
Damian Lillard         91.7  12   6    3  26  34:27  -5   4   0   4  
Pat Connaughton         0.0   0   1    0   3  18:31  -2   1   0   0  
Evan Turner             100   4   3    2  11  32:27  -4   7   0   1  
Ed Davis                100   4   1    7   6  16:02   4   0   2   0  
Shabazz Napier         50.0   2   1    0   3   6:07  -2   0   0   0  
Zach Collins             NA  NA       NA  NA    DNP  NA  NA  NA  NA  
Jake Layman              NA  NA       NA  NA    DNP  NA  NA  NA  NA  
Meyers Leonard           NA  NA       NA  NA    DNP  NA  NA  NA  NA  
Caleb Swanigan           NA  NA       NA  NA    DNP  NA  NA  NA  NA  
Khris Middleton         100   6   6    7  18  29:50   1   3   0   0  
Giannis Antetokounmpo  69.2  13  17    6  44  39:27   3   4   2   2  
Thon Maker             75.0   4   0    3   3  18:18  -6   2   1   1  
Tony Snell              0.0   0   7    1  17  34:29  -3   1   0   1  
Malcolm Brogdon        75.0   4   5    0  13  30:59   5   6   0   4  
DeAndre Liggins         0.0   0   0    1   0  18:24  10   0   1   0  
Mirza Teletovic         0.0   0   3    2   9  16:51   3   0   1   1  
Greg Monroe             0.0   0   0    0   0   6:44  -3   0   0   0  
Matthew Dellavedova     0.0   1   1    2   2  16:01  -1   4   0   0  
Jason Terry             0.0   0   0    0   0   4:59  -5   2   0   0  
John Henson            50.0   2   3    6   7  23:58  11   0   3   0  
Sterling Brown           NA  NA       NA  NA    DNP  NA  NA  NA  NA  
DJ Wilson                NA  NA       NA  NA    DNP  NA  NA  NA  NA

In [189]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv(games, header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [147]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#Setting up the index on the main stats dataframe
dateIndex = []
month = ['OCT', 'NOV', 'DEC', 'JAN', 'FEB']
for index, row in gamesDF.iterrows():
    for i in month:
        if gamesDF.iloc[index][0] == i:
            dateIndex.append(gamesDF.iloc[9,0] + " " + gamesDF.iloc[9,1][1] + ", " + gamesDF.iloc[9,2])
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

Home Away        W/L 1st Qtr H 2nd Qtr H 3rd Qtr H 4th Qtr H 1st Qtr A  \
0  POR  MIL  MILWAUKEE        26        29        25        30        24   

  2nd Qtr A 3rd Qtr A 4th Qtr A Total H Total A     Ref1       Ref2  \
0        36        24        29     113     110  Zarba""  Barnaky""   

           Ref3  
0  Voyard-Tadal

In [148]:
for index,row in finalStats.iterrows():
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [149]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)
finalStats = finalStats.fillna('NA')

In [150]:
finalStats['Player']

KeyError: 'Player'

In [156]:
len(newDict)

22

In [179]:
statsDF

PLAYER            MIN  FGM   FGA  FG%  3PM   3PA  3P%  FTM   FTA ...   \
0     Maurice       Harkless    F   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
1       37:50              5   10  50.0    2    4  50.0    1    2  50.0 ...    
2   Al-Farouq          Aminu    F   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
3       25:35              1    7  14.3    1    3  33.3    2    2   100 ...    
4       Jusuf         Nurkic    C   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
5       31:58              7   17  41.2    0    1   0.0    3    4  75.0 ...    
6          CJ       McCollum    G   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
7       37:03              9   18  50.0    3    6  50.0    5    5   100 ...    
8      Damian        Lillard    G   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
9       34:27              6   15  40.0    3    6  50.0   11   12  91.7 ...    
10        Pat    Connaughton  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
11      18:31              1    4  25.0    1    3  33.3    0    0   0.0 ...    
12       Evan         Turner  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
13      32:27              3    7  42.9    1    4  25.0    4    4   100 ...    
14         Ed          Davis  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
15      16:02              1    1   100    0    0   0.0    4    4   100 ...    
16    Shabazz         Napier  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
17       6:07              1    2  50.0    0    1   0.0    1    2  50.0 ...    
18       Zach        Collins  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
19       Jake         Layman  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
20     Meyers        Leonard  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
21      Caleb       Swanigan  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
22      Khris      Middleton    F   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
23      29:50              6   13  46.2    0    1   0.0    6    6   100 ...    
24    Giannis  Antetokounmpo    F   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
25      39:27             17   23  73.9    1    4  25.0    9   13  69.2 ...    
26       Thon          Maker    C   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
27      18:18              0    3   0.0    0    2   0.0    3    4  75.0 ...    
28       Tony          Snell    G   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
29      34:29              7    8  87.5    3    4  75.0    0    0   0.0 ...    
30    Malcolm        Brogdon    G   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
31      30:59              5   14  35.7    0    0   0.0    3    4  75.0 ...    
32    DeAndre        Liggins  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
33      18:24              0    2   0.0    0    1   0.0    0    0   0.0 ...    
34      Mirza      Teletovic  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
35      16:51              3    4  75.0    3    4  75.0    0    0   0.0 ...    
36       Greg         Monroe  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
37       6:44              0    3   0.0    0    0   0.0    0    0   0.0 ...    
38    Matthew    Dellavedova  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
39      16:01              1    5  20.0    0    2   0.0    0    1   0.0 ...    
40      Jason          Terry  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
41       4:59              0    0   0.0    0    0   0.0    0    0   0.0 ...    
42       John         Henson  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    
43      23:58              3    5  60.0    0    0   0.0    1    2  50.0 ...    
44   Sterling          Brown  NaN   NaN  NaN  NaN   NaN  NaN  NaN   NaN ...    

   OREB DREB  REB  AST  TOV  STL  BLK   PF  PTS  +/-  
0   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1     3    3    0    1    3    0    3   13    1  NaN  
2   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3     5    6    2    0    0    1    1    5   -1  NaN  
4   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  Na